In [2]:
import time
from datetime import datetime, timedelta, timezone

import pandas as pd
from sqlalchemy import create_engine
import pymysql
class QDP:

    def __init__(self, host, user, passward, database, port):
        """QDP SQL DATABASE

        Args:
            host (str): host ip
            user (str): user id
            passward (str): user passward
            database (str): name of database
            port (int): port

        Note:
            Database is a part of QDP project.
            It's a property of FBA Quant.
        """
        self.__sql_engine  = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}".format(
                                                                                                    user=user,
                                                                                                    pw=passward,
                                                                                                    host=host,
                                                                                                    port=port,
                                                                                                    db=database
                                                                                                )
                                        )                                       

    @property
    def table(self):
        command = "SHOW TABLE STATUS"
        df = pd.read_sql_query(sql=command, con=self.__sql_engine)
        return df
    
    @staticmethod
    def now(format="%Y:%m:%d %H:%M:%S.%f"):
        return datetime.fromtimestamp(time.time()).strftime(format)

    def get(self, table, symbols=None, columns=None, start=None, end=None, symbol_check=True):
        """Get Method

        Args:
            table (str): name of DB table
            symbols (list): list of symbols
            columns (list): list of columns
            start (DATE or DATETIME): %Y-%m-%d
            end (DATE or DATETIM): %Y-%m-%d
            symbol_check (bool): if True, checks whether the required symbols are in the table or not
        
        Returns:
            pd.DataFrame
        """

        # Manual limit of frequent queries... TODO --> 서버단위의 Query Limit 생성?
        time.sleep(0.25)

        # TypeChecking
        if type(symbols) != type(None) and type(symbols) != list:
            raise TypeError("symbols must be list")
        if type(columns) != type(None) and type(columns) != list:
            raise TypeError("columns must be list")

        # The code below returns Exception if the queried symbol doesn't exist in the TABLE
        if symbol_check:
            not_exist_symbol = []
            command = "SELECT DISTINCT SYMBOL FROM {table}".format(table=table)
            exist_symbol = set(pd.read_sql_query(sql=command, con=self.__sql_engine)["SYMBOL"])
            for symbol in symbols:
                if symbol not in exist_symbol:
                    not_exist_symbol.append(symbol)
            if len(not_exist_symbol) != 0:
                raise Exception("{} doesn't exist in {table}".format(not_exist_symbol, table=table))

        # Replace arguments for SQL query
        if type(symbols) == type(None):
            symbols = "*"
        else:
            symbols = str(symbols).replace("[","(").replace("]",")")

        if type(columns) == type(None):
            columns = "*"
        else:
            columns = ["DATE", "SYMBOL"] + columns
            columns = str(columns).replace("[","(").replace("]",")").replace("'","")

        if type(start) == type(None):
            start = self.now("%Y-%m-%d")
        if type(end) == type(None):
            end = self.now("%Y-%m-%d")

        # SQL Query
        command = "SELECT {columns} FROM {table} ".format(columns=columns, table=table)

        if symbols != "*":
            command = command + "WHERE SYMBOL in {symbols} ".format(symbols=symbols)
        
        if "WHERE" in command:
            command = command + "AND DATE BETWEEN '{start}' AND '{end}'".format(start=start, end=end)
        else:
            command = + "WHERE DATE BETWEEN '{start}' AND '{end}'".format(start=start, end=end)

        # Run the SQL command
        df = pd.read_sql_query(sql=command, con=self.__sql_engine)

        return df

In [3]:
# Initiate
qdp = QDP(
    host="39.123.37.56",
    user ="QDPUSER",
    passward="Fbaquant1#",
    database="QDP",
    port=3307
)

In [4]:
# Get data
price = qdp.get(
    table="PRICECHART_BY_DATE__KRX",
    symbols=["A005930"],
    start="2022-01-01",
    end="2023-01-01"
)

In [5]:
# Does it works well?
price

,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,VALUE
0,A005930,2022-01-03,79400,79800,78200,78600,13502112,1066006837750
1,A005930,2022-01-04,78800,79200,78300,78700,12427416,977990613000
2,A005930,2022-01-05,78800,79000,76400,77400,25470640,1967841509800
3,A005930,2022-01-06,76700,77600,76600,76900,12931954,996378432570
4,A005930,2022-01-07,78100,78400,77400,78300,15163757,1184236552700
...,...,...,...,...,...,...,...,...
241,A005930,2022-12-23,58200,58400,57700,58100,9829407,570401785680
242,A005930,2022-12-26,58000,58100,57700,57900,6756411,391223170600
243,A005930,2022-12-27,58000,58400,57900,58100,10667027,620410096800
244,A005930,2022-12-28,57600,57600,56400,56600,14665410,832919426100


In [6]:
qdp.table

,Name,Engine,Version,Row_format,Rows,Avg_row_length,Data_length,Max_data_length,Index_length,Data_free,Auto_increment,Create_time,Update_time,Check_time,Collation,Checksum,Create_options,Comment,Max_index_length,Temporary
0,PRICECHART_BY_DATE__KRX,InnoDB,10,Dynamic,7688439,89,689963008,0,0,5242880,None,2022-12-12 13:53:05,2023-02-17 18:30:47,None,utf8_general_ci,None,,,0,N
1,flyway_schema_history,InnoDB,10,Dynamic,0,0,16384,0,16384,0,None,2022-12-12 13:53:05,2022-12-12 13:53:07,None,utf8_general_ci,None,,,0,N


(a)

In [7]:
ts=datetime(2022,7,5).date() 
te=datetime(2022,7,11).date() 
price.loc[(price['DATE']>=ts) & (price['DATE']<=te)]

,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,VALUE
123,A005930,2022-07-05,57600,58200,57200,57200,14216539,818079785900
124,A005930,2022-07-06,57300,57300,56400,56400,16820461,955957046500
125,A005930,2022-07-07,56400,58700,56300,58200,21322833,1234677644065
126,A005930,2022-07-08,58600,59300,58200,58700,15339271,902198939736
127,A005930,2022-07-11,59300,59600,58700,58800,13042624,771042671500


In [8]:
datetime(2022,7,1).date()

datetime.date(2022, 7, 1)

In [17]:
price['sd']=0
price

,SYMBOL,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,VALUE,sd
0,A005930,2022-01-03,79400,79800,78200,78600,13502112,1066006837750,0
1,A005930,2022-01-04,78800,79200,78300,78700,12427416,977990613000,0
2,A005930,2022-01-05,78800,79000,76400,77400,25470640,1967841509800,0
3,A005930,2022-01-06,76700,77600,76600,76900,12931954,996378432570,0
4,A005930,2022-01-07,78100,78400,77400,78300,15163757,1184236552700,0
...,...,...,...,...,...,...,...,...,...
241,A005930,2022-12-23,58200,58400,57700,58100,9829407,570401785680,0
242,A005930,2022-12-26,58000,58100,57700,57900,6756411,391223170600,0
243,A005930,2022-12-27,58000,58400,57900,58100,10667027,620410096800,0
244,A005930,2022-12-28,57600,57600,56400,56600,14665410,832919426100,0


In [25]:
for idx,value in price.iterrows():
    print(value['CLOSE'])


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
pd.DataFrame

In [16]:
if None <1:
    print(a)

TypeError: '<' not supported between instances of 'NoneType' and 'int'